# Resonance Elastic Scattering of $\alpha$-particles with ${}^{10}\text{C}$

The GET acquisition system serialises data using the MultiFrame Metaformat (MFM)
> a binary format for data acquisition and serialization that are self-contained, layered, adapted to network transfers[,] and evolving. {cite:ps}`anvar_multiframe_nodate`

This file format is more suitable for disk storage than in-memory representation of data, as it is a heirarchical format in which each _frame_ must be read, regardless of whether it is needed. The use of separate header and data sections means that the cost of skipping a frame is limited to parsing the header, but MFM is still a struct-based format. In many cases, columnar processing is both more performant and more expressive than an equivalent event-loop based approach.{cite:ps}`smith_case_2019` 

The analysis process was performed in a series of discrete stages (see {numref}`analysis-flowchart`).

<!-- :::{figure-md} analysis-flowchart -->
```mermaid
flowchart LR

signal --> calibrate[Silicon Calibration*]
signal --> gain_matching[Gain Matching*]
gain_matching --> event
baseline_removal --> response[Response Estimation*]
calibrate --> kinematic_fitting
response --> signal

get[/GET MFM/] --> mfm2root[ROOT Conversion] --> partitioning[Partitioning] --> noise_removal[Noise Removal] --> baseline_removal[Baseline Removal] --> signal[Signal Fitting] --> event[Cluster Reconstruction] --> fit[Track Fitting] --> kinematic_fitting[Kinematic Fitting] --> kinematics[/Kinematics/]

```

<!-- The asterisk \* indicates processes which were aggregated over the entire dataset ahead of reconstruction -->
<!-- ::: -->

## Partitioning
At the earliest stages of the reconstruction pipeline, performance is primarily bounded by available memory, and therefore it is sensible to partition the data by event multiplicity. This process constitutes the first two stages of this analysis, seen in {numref}`analysis-flowchart`. In the ROOT dataset, the `mmMul` branch contains the per-event multiplicity. To regularise the dataset according to this multiplicity, it was partitioned into approximate chunks of a given aggregate multiplicity. Given that the memory of an MFM event is primarily determined by the number of waveforms that it contains, this effectively balances the memory footprint of each partition.

In subsequent stages, the memory bound gives way to compute costs, and so the partition size is refined to balance naive parallelisation against parallisation overhead. Where data is written to disk, the Hierarchical Data Format version 5 (HDF5) format is used as a consequence of its good language support, high compression ratios (with GZip), and columnar representation. The Parquet format is another strong format for columnar representation but, due to some library constraints, HDF5 was preferred.

## Noise and Baseline Removal

As discussed in {ref}`content:get-architecture`, the AGET chip provides four additional channels for measuring the electronic noise. These channels can be used to eliminate some of the electronic noise inherent within the system. A number of noise removal strategies are discussed in {cite:ps}`giovinazzo_get_2016-1`, and establishes that an average over the set of FPN channels per-chip is optimal. It was this strategy that was employed to for noise removal (see {numref}`fpn-removal-micromegas`).


:::{figure} todo
:name: fpn-removal-micromegas
:width: 400px
:align: center

```{warning}
Todo: add example of FPN channel & FPN removal
```
:::

In addition to a low frequency component of the electronic noise, a separate baseline offset was observed in the MicroMeGAS and silicon detector channels. This component ws removed using two separate methods.

### MicroMeGaS Baseline Estimation

Given that time in the MicroMeGaS is primarly determined by the drift time of the liberated electrons in the gas volume, the baseline observed in the MicroMeGaS detector was not seen to occur at specific times. Therefore, in order to identify the region of baseline that would be linearly subtracted from the recorded signal, a local variance was computed for the channel samples, with a window of 96 cells (see {numref}`baseline-estimate-micromegas`).

:::{figure} todo
:name: baseline-estimate-micromegas
:width: 400px
:align: center

```{warning}
Todo: add example of baseline estimation
```
:::


The value of the baseline was given by the locus of the minimum computed variance.

### Silicon Baseline Estimation

Unlike the MicroMeGaS channels, the silicon quadrant detectors have a well defined boundary between the baseline and the interaction signals. This boundary occurs at a similar time for each detector, as it is the silicon detectors upon which the GET system was configured to trigger. To determine the baseline value, the mean of the sample values for the preceeding ten cells was used (see {numref}`baseline-estimate-silicon`).

:::{figure} todo
:name: baseline-estimate-silicon
:width: 400px
:align: center

```{warning}
Todo: add example of baseline estimation
```
:::

## Response Estimation

Channels which employ the AGET shaper require deconvolution in order to recover the underlying signal. This process requires a good knowledge of the underlying response function of the system. An approximate analytical solution is given in {cite:ps}`giovinazzo_get_2016-1`, but the authors note that it is insufficiently accurate to be used with deconvolutional methods. It is noted that the electronics can be configured such that the response function can be directly measured, but this required secondary access to the detector which resides at the Cyclotron Institute, Texas A&M University.{cite:ps}`giovinazzo_get_2016-1` 

In order to determine a better approximation for the response function, the silicon waveforms recorded by the electronics can be used. In particular, it must be assumed that the shape of the current signal produced by interactions with the silicon detector is predominantly determined by the shaper. With this assumption, the silicon-derived response function can be used to deconvolve MicroMeGaS signals, whose widths are determined by the ionisation within the gas volume. 

:::{warning}
Figure comparing widths of MM and Si detectors
:::

To build a response function estimate, a random sample of silicon waveforms is taken 

## Signal Fitting

## Gain Matching

The Micromegas anode is subdivided into a set of distinct zones that can be held at different potentials in order to spatially vary the gain. In this experiment, both the strip-chain (side) regions and final block of pads in the central region were held at a high gain in order to better resolve light particle tracks with low stopping powers. Following position reconstruction, these gains must be accounted for if the collected charge is to be used for particle identification.

Given that the strips, chains, and final central pads were held at the same potential, the relative gain of these regions can be determined solely by looking at the relative gain of the final central pad region. This was experimentally measured by observing the change in charge collected by the rows of pads either side of the high-low gain boundary.

:::{note}
TODO: add an image of MM gain boundary
:::

```{code-cell} python
def compute_gain_thick_hist(chan):
    # Compute the max of the sample waveform
    charge = np.max(chan.sample, axis=-1)

    # Identify which events have a cross-boundary track
    is_before = (chan.addr.index.chan >= 448) & (chan.addr.index.chan <= 451)
    is_after = (chan.addr.index.chan >= 252) & (chan.addr.index.chan <= 255)
    crosses_boundary = np.any(is_before, axis=-1) & np.any(is_after, axis=-1)
    
    # Compute ratio of collected charges
    factor = (
        np.sum(
            charge[crosses_boundary & is_before], axis=-1
        ) / 
        np.sum(
            charge[crosses_boundary & is_after], axis=-1
        )

    return Hist.new.Reg(128, 0, 1).Int64().fill(np.ravel(factor))
```

```{figure} image/gain-high-dark.png
---
alt: Ratio of low gain and high gain mean charge at the gain boundary.
class: only-dark
figclass: margin
width: 400px
align: center
---
Ratio of the mean collected charge between the low gain and high gain region boundary in the Micromegas pads.
```

## Silicon Calibration

## Beam

## Garfield++ Simulation